In [182]:
import requests
from bs4 import BeautifulSoup
import gspread
# from oauth2client.service_account import ServiceAccountCredentials
import firebase_admin
from firebase_admin import credentials, storage
import os
from urllib.parse import urlparse
import re
import unicodedata

In [183]:
url = "https://www.childrensplace.com/us/p/Girls-Long-Sleeve-Open-Front-Cardigan-3049648-327S"

In [184]:
# response = requests.get(url).text
with open("input.html", 'r', encoding='utf-8') as file:
        response = file.read()

In [185]:
def create_slug(text):
    # Convert the text to lowercase
    text = text.lower()

    # Remove non-word characters (everything except numbers and letters)
    text = re.sub(r'[^\w\s-]', '', text)

    # Replace all runs of whitespace with a single dash
    text = re.sub(r'\s+', '-', text)

    # Remove leading/trailing dashes
    text = text.strip('-')

    # Handle unicode characters
    text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('ascii')

    return text

In [186]:
# print(response)
soup = BeautifulSoup(response, 'html.parser')

# print(soup)

try:
    # Modify these selectors based on the website's structure
    product_name = soup.select_one('h1.product-title').text.strip()
    product_price = soup.select_one('p.actual-price').text.strip().replace("$", "")
    product_image_url = soup.select_one('div.product-image-wrapper.product-wrapper-inline source')['srcset']
    product_desc = soup.select_one('div.product-description-list li').text.strip()

    # image_url = product_image_url.split(" ")[-2]
    # image_url = product_image_url.split(" ")[2]
    image_url = product_image_url.split(" ")[0]
    slug = create_slug(product_name)
    image_name = f"{slug}.png"
    print(image_url)
    print(slug)
    # product_description = soup.select_one('CSS_SELECTOR_FOR_PRODUCT_DESCRIPTION').text.strip()
    # product_image_url = soup.select_one('CSS_SELECTOR_FOR_PRODUCT_IMAGE')['src']
except Exception as e:
    print(e)

🚀 ~ product_desc: Made of 100% polyester flannel
https://assets.theplace.com/image/upload/t_pdp_img_d,f_auto,q_auto/v1/ecom/assets/products/tcp/3023881/3023881_01.png
1x,
https://assets.theplace.com/image/upload/t_pdp_img_d,f_auto,q_auto,dpr_1.5/v1/ecom/assets/products/tcp/3023881/3023881_01.png
2x,
https://assets.theplace.com/image/upload/t_pdp_img_d,f_auto,q_auto,dpr_3.0/v1/ecom/assets/products/tcp/3023881/3023881_01.png
3x
https://assets.theplace.com/image/upload/t_pdp_img_d,f_auto,q_auto/v1/ecom/assets/products/tcp/3023881/3023881_01.png
unisex-kids-plaid-flannel-pajamas---black


In [187]:
id = "1O_MJRgc67xHE4T6ATfzCBLKqmLMirWmwBIZwRh5y6Jg"

In [188]:

# Set up credentials (you'll need to create a service account and download the JSON key)
# scope = ['https://spreadsheets.google.com/feeds', 'https://www.googleapis.com/auth/drive']
# creds = ServiceAccountCredentials.from_json_keyfile_name('./shopit.json', scope)
# client = gspread.authorize(creds)

gc = gspread.service_account(filename='shopit.json')
# gc = gspread.oauth()


# Open the Google Sheet (replace with your sheet ID)
sht2 = gc.open_by_key(id).sheet1

sht2.append_row([product_name, slug, product_desc, float(product_price) * 1500, 0, image_name, 4.9])


{'spreadsheetId': '1O_MJRgc67xHE4T6ATfzCBLKqmLMirWmwBIZwRh5y6Jg',
 'tableRange': 'products!A1:G42',
 'updates': {'spreadsheetId': '1O_MJRgc67xHE4T6ATfzCBLKqmLMirWmwBIZwRh5y6Jg',
  'updatedRange': 'products!A43:G43',
  'updatedRows': 1,
  'updatedColumns': 7,
  'updatedCells': 7}}

In [189]:
# Initialize Firebase app (you'll need to set up a Firebase project and download the credentials)
cred = credentials.Certificate('firebase-credentials.json')
if not firebase_admin._apps:
    firebase_admin.initialize_app(cred, {'storageBucket': 'shopit-ebc60.appspot.com'})

# Download the image
response = requests.get(image_url)
# image_name = f"{product_name}.jpg"
with open(image_name, 'wb') as f:
    f.write(response.content)

# Upload to Firebase Storage
bucket = storage.bucket()
blob = bucket.blob(f"products/{image_name}")
blob.upload_from_filename(image_name)

# Clean up the local file
os.remove(image_name)